# 1. Description

This notebook will reproduce Figure 3 in the paper "Prediction of the ICU mortality based on the missing events.".

# 2. Before running...

Before proceeding the followings, plaease solve the python environment accordingly first. This program requires the following libraries.

In [2]:
from math import sqrt
import pandas as pd
import numpy as np
import itertools

# sklearn 0.24.1
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import calibration_curve
from sklearn import metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Bokeh 2.2.3
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.models.annotations import Label                  
from bokeh.models import Legend
from bokeh.models import LinearAxis, Range1d

Then please put the related files at the appropriate directory so that the program reaches the input files. To test if you correctly set the input files, run the below. If you could, the cell would not end without errors.

In [3]:
input_ids = set(pd.read_csv("../ids/004_sepsis_aps_4226.csv", header=None).iloc[:,0].tolist())
len(input_ids) # 4226

4226

In [4]:
eICU_file = {}
eICU_file['apacheApsVar'] = pd.read_csv('../data/apacheApsVar.csv')
eICU_file['apachePatientResult'] = pd.read_csv('../data/apachePatientResult.csv')
eICU_file['apachePredVar'] = pd.read_csv('../data/apachePredVar.csv')
eICU_file['patient'] = pd.read_csv('../data/patient.csv')

# 3. Custom classes and functions

For the detail, see the notebook "2_subgrouping_sepsis.ipynb", "3_subgrouping_non_sepsis.ipynb" or "4_sepsis_prediction.ipynb".

## 3.1 Class for subgrouping

In [5]:
class status():
    def __init__(self, df):
        # df
        self.df = df
        self.df_thistime = pd.DataFrame([], columns=df.columns)
        self.df_next = df
        # ID
        self.ids_all = set(df.patientunitstayid)
        self.ids_thistime = set([])
        self.ids_next = self.ids_all
        # parameter
        self.target = []
        
        
    def remove(self, target):
        self.target += target
        # Update ID
        tmp = set(self.df_next.drop(target, axis=1).where(self.df_next>=0).dropna().patientunitstayid)
        self.ids_thistime |= tmp
        self.ids_next -= tmp
        
        # df thistime
        self.df_thistime = self.df.drop(self.target, axis=1)
        self.df_thistime = self.df_thistime.query("patientunitstayid in @ self.ids_thistime")
        
        # df_next
        self.df_next = self.df_next.drop(target, axis=1)
        self.df_next = self.df_next.query("patientunitstayid in @ self.ids_next")
        
        
    def get_next(self, depth):
        # 1st and Last columns are not paramters
        parameters = self.df_next.columns[1:-1]
        
        # depth : the number of parameters to be excluded at once
        combinations = [list(i) for i in itertools.combinations(parameters, depth)]
        
        # # of non-NaN-records 
        num_non_nan = pd.DataFrame({
            "__".join(comb) : [len(self.df_next.drop(comb, axis=1).where(self.df_next>=0).dropna())]
            for comb in combinations
        }).T
        
        # "1 <= # of non-NaN-records < # of pids" is ideal.
        tf = num_non_nan.applymap(lambda x : 1 <= x <= len(self.df_next) - 1)        
        if tf.any().any():
            tmp = num_non_nan[tf]
            return tmp.idxmax()[0]
        
        else:
            # "# of non-NaN-records == # of pids" is acceptable.
            tf = num_non_nan.applymap(lambda x : 1 <= x <= len(self.df_next))
            if tf.any().any():
                tmp = num_non_nan[tf]
                return tmp.idxmax()[0]
            
            else:
                # If there's no more parameteres, return nan      
                if len(parameters) == 1:
                    return "nan"

                # If there's only NaN records, return ""
                else:
                    return ""

## 3.2. Functions for preparing "Training Data and Test Data"

In [6]:
def pid_train_test_split(df_A):
    # df_A  -->  pid_X and y
    length = len(df_A.columns)
    pid_X = df_A.iloc[:, : length-1].values
    y = df_A.iloc[:, -1].values

    pid_X_train, pid_X_test, y_train, y_test = split_stratified(pid_X, y)

    length = pid_X_train.shape[1]
    X_train = pid_X_train[:, 1:length]
    X_test = pid_X_test[:, 1:length]
    pid_train = pid_X_train[:, 0]
    pid_test = pid_X_test[:, 0]
    
    # Return as Dictionary
    dataset = {
        "pid_train" : pid_train,
        "pid_test" : pid_test,
        "X_train" : X_train,
        "X_test" : X_test,
        "y_train" : y_train,
        "y_test" : y_test
    }    
    return dataset



def split_stratified(pid_X, y):
    skfold = StratifiedKFold(n_splits=5, shuffle=True)
    index = [i for i in skfold.split(pid_X, y)]
    train_index = index[0][0]
    test_index = index[0][1]
    pid_X_train = pid_X[train_index,:]
    pid_X_test = pid_X[test_index,:]
    y_train = y[train_index]
    y_test = y[test_index]
    return pid_X_train, pid_X_test, y_train, y_test

## 3.3. Functions for the Random Forest

In [7]:
def grid_search(X_train, y_train):
    skfold = StratifiedKFold(n_splits=5, shuffle=True)
    h_parms={
        "n_estimators" : [i for i in range(10,50,10)],
        "criterion" : ["gini","entropy"],
        "max_depth" : [i for i in range(1,6,1)],
        "random_state" : [123, 777, 2525]
    }
    rdm_gs = GridSearchCV(RandomForestClassifier(), param_grid=h_parms, cv=skfold, n_jobs=5)    
    rdm_gs.fit(X_train, y_train)
    h_parm = rdm_gs.best_estimator_.get_params(True).items()
    h_parm = {tup[0] : tup[1] for tup in h_parm}
    return h_parm


def cross_validation(h_parm, X_train, y_train):
    # Model
    rdm = RandomForestClassifier(
        n_estimators = h_parm["n_estimators"], 
        criterion = h_parm["criterion"],
        max_depth = h_parm["max_depth"], 
        random_state = h_parm["random_state"]
    )        
    # Split Criteria
    skfold = StratifiedKFold(n_splits=5, shuffle=True)
    # Cross Validation
    scores = cross_val_score(rdm, X_train, y_train, cv=skfold, scoring="accuracy")
    # Print Mean & STD
    score_mean = np.mean(scores)
    score_std = np.std(scores)    
    print(str(round(score_mean,5)) + ' ± ' + str(round(score_std,5)))
    
    
def fitting(h_parm, X_train, y_train):
    rdm = RandomForestClassifier(
        n_estimators = h_parm["n_estimators"], 
        criterion = h_parm["criterion"],
        max_depth = h_parm["max_depth"], 
        random_state = h_parm["random_state"]
    )
    rdm.fit(X_train, y_train)      
    return rdm


def prediction(rdm, X_test):
    y_predicted = rdm.predict_proba(X_test)[:, 1]
    return y_predicted

## 3.4. Functions related to drawing the ROCs.

In [8]:
def roc_auc(y_test, y_predicted, y_test_IV, y_predicted_IV, y_test_IVa, y_predicted_IVa):
    
    # Ours
    fpr_ours, tpr_ours, thresholds = metrics.roc_curve(y_test, y_predicted, drop_intermediate=False)
    auc_ours = metrics.auc(fpr_ours, tpr_ours)

    # APACHE IV
    fpr_IV, tpr_IV, thresholds = metrics.roc_curve(y_test_IV, y_predicted_IV, drop_intermediate=False)
    auc_IV = metrics.auc(fpr_IV, tpr_IV)    

    # APACHE IVa
    fpr_IVa, tpr_IVa, thresholds = metrics.roc_curve(y_test_IVa, y_predicted_IVa, drop_intermediate=False)
    auc_IVa = metrics.auc(fpr_IVa, tpr_IVa)    

    # Ours + APACHE IV + APACHE IVa dataset for Bokeh
    source = ColumnDataSource(data=dict(
        x_ours = fpr_ours,
        y_ours = tpr_ours,    
        x_IV = fpr_IV,
        y_IV = tpr_IV,    
        x_IVa = fpr_IVa,
        y_IVa = tpr_IVa,    
    ))
    
    # Title, Size
    p = figure(
        title = 'ROC curve',
        plot_width=350,
        plot_height=350
    )
    
    # Axis Labels
    p.xaxis.axis_label = 'False Positive Rate'
    p.yaxis.axis_label = 'True Positive Rate'

    # Plot
    p.line('x_ours', 'y_ours', color='red', legend_label='Ours', line_width=2, source=source)
    p.line('x_IV', 'y_IV', color='green', legend_label='APACHE IV', line_width=2, source=source)
    p.line('x_IVa', 'y_IVa', color='blue', legend_label='APACHE IVa', line_width=2, source=source)

    # Legend
    p.legend.location = "bottom_right"
    p.legend.label_text_font_style = "bold"
    #p.legend.label_text_font_size = "1pt"
    
    # Texts for AUC
    # "AUC"
    description=Label(
        x_offset=145,
        y_offset=145,
        x_units='screen',
        y_units='screen',
        text = 'AUC',
        text_font_size='1pt', render_mode="css"
    )        
    p.add_layout(description)    

    # AUC for Ours
    description=Label(
        x_offset=140,
        y_offset=130,
        x_units='screen',
        y_units='screen',
        text = '%.5f (Ours)'%auc_ours,
        text_font_size='1pt', render_mode="css"
    )        
    p.add_layout(description)    

    # AUC for APACHE IV
    description=Label(
        x_offset=140,
        y_offset=115,
        x_units='screen',
        y_units='screen',
        text = '%.5f (APACHE IV)'%auc_IV,
        text_font_size='1pt', render_mode="css"
    )        
    p.add_layout(description)    

    # AUC for APACHE IVa
    description=Label(
        x_offset=140,
        y_offset=100,
        x_units='screen',
        y_units='screen',
        text = '%.5f (APACHE IVa)'%auc_IVa,
        text_font_size='1pt', render_mode="css"
    )        
    p.add_layout(description)    
    
    # Output on the notebook
    output_notebook()    
    show(p)
    
    a = str(round(auc_ours,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_ours, np.array(y_test), np.array(y_predicted))])+"]"
    b = str(round(auc_IV,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_IV, np.array(y_test_IV), np.array(y_predicted_IV))])+"]"
    c = str(round(auc_IVa,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_IVa, np.array(y_test_IVa), np.array(y_predicted_IVa))])+"]"
    return [[a, b, c]]


def get_ci(auc, y_test, y_predicted):
    n1 = sum(y_test==1)
    n2 = sum(y_test!=1)
    q1 = auc / (2- auc)
    q2 = 2*auc**2 / (1+auc)
    se_auc = sqrt((auc*(1 - auc) + (n1 - 1)*(q1 - auc**2) + (n2 - 1)*(q2 - auc**2)) / (n1*n2))
    lower = auc - 1.96*se_auc
    upper = auc + 1.96*se_auc
    lower = 0 if lower <0 else lower
    upper = 1 if upper >1 else upper
    return lower, upper

## 3.5. Functions for the scatterred plots.

In [9]:
def plot_check(df_p):
    # DataFrame
    apache_IV = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IV"][['patientunitstayid', 'predictedicumortality','actualicumortality']]
    apache_IVa = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IVa"][['patientunitstayid', 'predictedicumortality']]
    df_IV_IVa = pd.merge(apache_IV, apache_IVa, on="patientunitstayid")
    df_p_IV_IVa = pd.merge(df_IV_IVa, df_p, on="patientunitstayid")
    
    # Graph title
    graph_title = ["IV vs IVa", "IV vs Ours", "IVa vs Ours"]

    # Axis labels
    x_axis_label = ["APACHE IV", "APACHE IV", "APACHE IVa"]
    y_axis_label = ["APACHE IVa", "Ours", "Ours"]

    # Graph data
    x_data = [df_p_IV_IVa.predictedicumortality_x, df_p_IV_IVa.predictedicumortality_x, df_p_IV_IVa.predictedicumortality_y]
    y_data = [df_p_IV_IVa.predictedicumortality_y, df_p_IV_IVa.predictedicumortality, df_p_IV_IVa.predictedicumortality]
    
    # patientunitstayid
    pid = df_p_IV_IVa.patientunitstayid
    
    colormap = {'ALIVE': 'green', 'EXPIRED': 'red'}
    df_p_IV_IVa['color'] = df_p_IV_IVa['actualicumortality'].map(lambda x: colormap[x])

    
    output_notebook()

    p=[]
    p.append([])
    
    for i in range(3):
                    
        # Other configurations
        x_size = 300
        y_size = 300
        alpha = 0.2
        size = 4

        # graph objects
        source = ColumnDataSource(data=dict(
            x=x_data[i],
            y=y_data[i], 
            pid=pid,
            color=df_p_IV_IVa['color'],
            legend=df_p_IV_IVa['actualicumortality']
        ))
                
        TOOLTIPS = [("index", "$index"),("(x,y)", "($x, $y)"),("pid", "@pid")]

        p[-1].append(figure(
            title = graph_title[i],
            plot_width = x_size,
            plot_height = y_size,
            tooltips = TOOLTIPS,
            x_range=(0, 1),
            y_range=(0, 1)
        ))
        
        p[-1][-1].xaxis.axis_label = x_axis_label[i]
        p[-1][-1].yaxis.axis_label = y_axis_label[i]
        p[-1][-1].circle(
            'x', 'y',
            fill_color="color",
            line_color="color",
            fill_alpha=alpha,
            legend_label='legend',
            size=size,
            source=source
        )
                
        p[-1][-1].legend.location = "top_left"
        p[-1][-1].legend.label_text_font_style = "bold"
            
    grid = gridplot(p)
    show(grid)

## 3.6. Functions for the Calibration Plots

In [10]:
def calibration_plot(df_p):
    # DataFrame
    apache_IV = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IV"][['patientunitstayid', 'predictedicumortality','actualicumortality']]
    apache_IVa = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IVa"][['patientunitstayid', 'predictedicumortality']]
    df_IV_IVa = pd.merge(apache_IV, apache_IVa, on="patientunitstayid")
    df_p_IV_IVa = pd.merge(df_IV_IVa, df_p, on="patientunitstayid")
    df_p_IV_IVa = df_p_IV_IVa.where(df_p_IV_IVa>=0).dropna()
    
    # Probability from Ours, IV and IVa
    probs = [df_p_IV_IVa.predictedicumortality, df_p_IV_IVa.predictedicumortality_x, df_p_IV_IVa.predictedicumortality_y]

    # Outcome
    numbermap = {'ALIVE': 0, 'EXPIRED': 1}
    outcome = df_p_IV_IVa['actualicumortality'].map(lambda x: numbermap[x])

    # Graph title
    graph_title = ["Ours", "IV", "IVa"]
    
    output_notebook()

    p=[]
    
    for i in range(3):

        p.append([])
                            
        # Data for Calibration Plot
        prob_true, prob_pred = calibration_curve(y_true=outcome, y_prob=probs[i], n_bins=10)

        # graph objects                
        p[-1].append(figure(
            title = graph_title[i],
            plot_width = 600,
            plot_height = 300,
            y_range=(0, 1.05)
        ))

        # Axis
        p[-1][-1].xaxis.axis_label = "Predicted Probability"
        p[-1][-1].yaxis.axis_label = "Actual Probability"

        # Plot              
        p1 =p[-1][-1].line(prob_pred, prob_true, line_width=2)
        p2 = p[-1][-1].circle(prob_pred, prob_true, fill_color="white", size=4)

        # Ideal
        p3 = p[-1][-1].line([0,1], [0,1], line_color="red", line_dash="dotted", line_width=2)
        
        # histogram
        hist, edges = np.histogram(probs[i], bins=10)
        p[-1][-1].extra_y_ranges = {"hist": Range1d(start=0, end=max(hist))}
        linaxis = LinearAxis(axis_label="# of cases", y_range_name='hist')
        p[-1][-1].add_layout(linaxis, 'right')
        p[-1][-1].quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color="navy", line_color="white", alpha=0.5, y_range_name="hist")
        
        
        # Legend        
        legend = Legend(items=[
            ("Calibration"   , [p1, p2]),
            ("Ideal" , [p3])
        ], location="bottom_right", label_text_font_style = "bold")        
        p[-1][-1].add_layout(legend, 'right')
            
    grid = gridplot(p)
    show(grid)
    

# 4. Prepare DataFrame

For the detail, see the notebook "2_subgrouping_sepsis.ipynb", "3_subgrouping_non_sepsis.ipynb" or "4_sepsis_prediction.ipynb".

## 4.1. Definition of variables used in this study

In [11]:
eICU_parm = {}

eICU_parm['apacheApsVar'] = [
    'patientunitstayid',
    'intubated',
    'vent',
    'dialysis',
    'eyes',
    'motor',
    'verbal',
    'meds',
    'urine',
    'wbc',
    'temperature',
    'respiratoryrate',
    'sodium',
    'heartrate',
    'meanbp',
    'ph',
    'hematocrit',
    'creatinine',
    'albumin',
    'pao2',
    'pco2',
    'bun',
    'glucose',
    'bilirubin',
    'fio2'
]

eICU_parm['apachePatientResult'] = [
    'patientunitstayid',
    'apachescore',
    'predictedicumortality',
    'predictediculos',
    'predictedhospitalmortality',
    'predictedhospitallos',
    'preopmi',
    'preopcardiaccath',
    'ptcawithin24h',
    'predventdays'
]

eICU_parm['apachePredVar'] = [
    'patientunitstayid',
    'gender',
    'teachtype',
    'bedcount',
    'graftcount',
    'age',
    'thrombolytics',
    'aids',
    'hepaticfailure',
    'lymphoma',
    'metastaticcancer',
    'leukemia',
    'immunosuppression',
    'cirrhosis',
    'ima',
    'midur',
    'ventday1',
    'oobventday1',
    'oobintubday1',
    'diabetes'
]

eICU_parm['patient'] = [
    'patientunitstayid',
    'hospitalid',
    'admissionheight',
    'hospitaladmitoffset',
    'admissionweight'
]

## 4.2. DataFrame

In [12]:
#========================================
#  select columns and ids for each file
#========================================
eICU_df = {}
eICU_df['apacheApsVar'] = eICU_file['apacheApsVar'][eICU_parm['apacheApsVar']].query("patientunitstayid in @ input_ids") 
eICU_df['apachePatientResult'] = eICU_file['apachePatientResult'][eICU_parm['apachePatientResult']].query("patientunitstayid in @ input_ids") 
eICU_df['apachePredVar'] = eICU_file['apachePredVar'][eICU_parm['apachePredVar']].query("patientunitstayid in @ input_ids") 
eICU_df['patient'] = eICU_file['patient'][eICU_parm['patient']].query("patientunitstayid in @ input_ids") 


#========================================
#  make column names unique
#========================================
#  (column name -> filename + column name)
eICU_df['apacheApsVar'].columns = [
    'apacheApsVar_' + parm if not parm=="patientunitstayid" else "patientunitstayid"
    for parm in eICU_df['apacheApsVar'].columns
]
eICU_df['apachePatientResult'].columns = [
    'apachePatientResult_' + parm if not parm=="patientunitstayid" else "patientunitstayid"
    for parm in eICU_df['apachePatientResult'].columns
]
eICU_df['apachePredVar'].columns = [
    'apachePredVar_' + parm if not parm=="patientunitstayid" else "patientunitstayid"
    for parm in eICU_df['apachePredVar'].columns
]
eICU_df['patient'].columns = [
    'patient_' + parm if not parm=="patientunitstayid" else "patientunitstayid"
    for parm in eICU_df['patient'].columns
]


#========================================
#  Make X
#========================================
# 1st column : key (patientunitstayid)
key = pd.DataFrame(list(input_ids), columns=["patientunitstayid"])

# 2nd~ column : parameters
key_X = pd.merge(key, eICU_df['apacheApsVar'], on="patientunitstayid")
key_X = pd.merge(key_X, eICU_df['apachePatientResult'], on="patientunitstayid")
key_X = pd.merge(key_X, eICU_df['apachePredVar'], on="patientunitstayid")
key_X = pd.merge(key_X, eICU_df['patient'], on="patientunitstayid")


#========================================
#  Make X_y (df)
#========================================
# Last column : DEAD(=1) or ALIVE(=0)
y = eICU_file["apachePatientResult"][['patientunitstayid', 'actualicumortality']].replace('ALIVE',0).replace('EXPIRED',1)
key_X_y = pd.merge(key_X, y, on="patientunitstayid")


#========================================
# Rename
#========================================
df = key_X_y

# 5. Generating our model

For the detail, see the notebook "2_subgrouping_sepsis.ipynb", "3_subgrouping_non_sepsis.ipynb" or "4_sepsis_prediction.ipynb".

In [13]:
df_status = status(df)
k=1

t_y_test, t_y_predicted, t_y_test_IV, t_y_predicted_IV, t_y_test_IVa, t_y_predicted_IVa = [],[],[],[],[],[]
t_pid_test, t_y_predicted = [], []
df_ci = []

print("# of INPUT : ", len(df_status.ids_all), " patientunitstayids", "\n\n")

while 1:
    print("##################################")
    print("                          Subgroup ",k) 
    print("##################################")
    print("\n") 
    print("Checking the inputs...")
    print("\n") 

    
    
    parms_tobe_excluded = []
    
    #========================================
    #  Get Subgroup
    #========================================
    while not(500 <= len(df_status.ids_thistime) <= 2000):

        parms = ""

        # upto 3 parameters taken into account
        for i in range(1,4):
            parms = df_status.get_next(i)

            # if parameters are found
            if parms != "":
                break
                

        # If no paramteres found, Output "Time out" and Stop.
        if parms == "":
            print("Time Out\n")
            df_status.df_next = pd.DataFrame()
            break
            
        # If too many nan, Output "Interpolation needed" and Stop
        if parms == "nan":
            print("Interpolation needed\n")
            df_status.df_next = pd.DataFrame()
            break

            
        # Change format
        parms = parms.split("__")        

        # Update
        df_status.remove(parms)
        parms_tobe_excluded += parms
        print("--> ", [i.split("_")[1] for i in parms], " is/are selected to be excluded.\n")
        
  

    print("--> ", ", ".join([i.split("_")[1] for i in parms_tobe_excluded]), " was/were excluded in the end.\n")
    parms_tobe_excluded = []

    
    
    df_A = pd.DataFrame()
    
    if 500 <= len(df_status.ids_next):
        print("--> ", len(df_status.ids_thistime), " patientunitstayids survived.\n")
        df_A = df_status.df_thistime
        df_status = status(df_status.df_next)
        
    else:
        # The rests pids are picked up and merged into thistime.
        print("--> ", len(df_status.ids_thistime)+len(df_status.ids_next), " patientunitstayids survived.\n")
        df_A = pd.concat([df_status.df_thistime, df_status.df_next])
        df_status.ids_next = set([])
        df_status.df_next = df_status.df_next.query("patientunitstayid in @ df_status.ids_next")
        
    df_A = df_A.where(df_A>=0).dropna()

    #========================================
    #  Learning
    #========================================    
    # Prepare Trainind data and Test Data
    dataset = pid_train_test_split(df_A)
    X_train = dataset["X_train"]
    y_train = dataset["y_train"]
    X_test = dataset["X_test"]
    y_test = dataset["y_test"]
    pid_test = dataset["pid_test"]

    # Grid Search
    h_parm = grid_search(X_train, y_train)
    
    # Cross Validation
    cross_validation(h_parm, X_train, y_train)

    # Model Generation
    rdm = fitting(h_parm, X_train, y_train)

    # Prediction
    y_predicted = prediction(rdm, X_test)

    
    #========================================
    #  Evaluation
    #========================================    
    # Prediction by APACHE
    df_apache = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IV"].query("patientunitstayid in @ pid_test")
    id_IV = df_apache[df_apache["apacheversion"]=="IV"]['patientunitstayid'].values.tolist()
    y_test_IV = df_apache.query("patientunitstayid in @ id_IV").actualicumortality.replace('ALIVE',0).replace('EXPIRED',1).values.tolist()
    y_predicted_IV = df_apache.query("patientunitstayid in @ id_IV").predictedicumortality.values.tolist()

    df_apache = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IVa"].query("patientunitstayid in @ pid_test")
    id_IVa = df_apache[df_apache["apacheversion"]=="IVa"]['patientunitstayid'].values.tolist()
    y_test_IVa = df_apache.query("patientunitstayid in @ id_IVa").actualicumortality.replace('ALIVE',0).replace('EXPIRED',1).values.tolist()
    y_predicted_IVa = df_apache.query("patientunitstayid in @ id_IVa").predictedicumortality.values.tolist()
    
    
    # Store subgroup information
    t_y_test += list(y_test)
    t_y_predicted += list(y_predicted)
    t_y_test_IV += list(y_test_IV)
    t_y_predicted_IV += list(y_predicted_IV)
    t_y_test_IVa += list(y_test_IVa)
    t_y_predicted_IVa += list(y_predicted_IVa)
    t_pid_test += pid_test.tolist()
    
    
    # ROC and AUC
    df_ci += roc_auc(y_test, y_predicted, y_test_IV, y_predicted_IV, y_test_IVa, y_predicted_IVa)
        
    # Comparison with APACHE
    df_p = pd.DataFrame([pid_test.tolist(), y_predicted.tolist()]).T
    df_p.columns = ["patientunitstayid","predictedicumortality"]
    plot_check(df_p)

    # Calibration Plots
    calibration_plot(df_p)
    
    k+=1
    
    if len(df_status.df_next) ==  0:
        break

        
        
#========================================
#  Integration
#========================================        
print("##################################")
print("                          Overall (Integrated)") 
print("##################################")

# ROC and AUC
df_ci += roc_auc(t_y_test, t_y_predicted, t_y_test_IV, t_y_predicted_IV, t_y_test_IVa, t_y_predicted_IVa)

# Comparison with APACHE
df_p = pd.DataFrame([t_pid_test, t_y_predicted]).T
df_p.columns = ["patientunitstayid","predictedicumortality"]
plot_check(df_p)

# Calibration Plots
calibration_plot(df_p)


# Confidence Interval
df_ci = pd.DataFrame(df_ci, columns=["Ours","Apache IV","Apache IVa"], index=["1","2","3","4","5","ALL"])
df_ci

# of INPUT :  4226  patientunitstayids 


##################################
                          Subgroup  1
##################################


Checking the inputs...


-->  ['hospitaladmitoffset']  is/are selected to be excluded.

-->  hospitaladmitoffset  was/were excluded in the end.

-->  884  patientunitstayids survived.

0.85078 ± 0.01153


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

##################################
                          Subgroup  2
##################################


Checking the inputs...


-->  ['urine']  is/are selected to be excluded.

-->  urine  was/were excluded in the end.

-->  1037  patientunitstayids survived.

0.85353 ± 0.00448


...

##################################
                          Overall (Integrated)
##################################


Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

,Ours,Apache IV,Apache IVa
1,"0.902 [0.87,0.935]","0.745 [0.691,0.799]","0.741 [0.687,0.795]"
2,"0.881 [0.849,0.913]","0.753 [0.705,0.801]","0.749 [0.701,0.797]"
3,"0.953 [0.909,0.996]","0.731 [0.628,0.834]","0.733 [0.63,0.836]"
4,"0.895 [0.862,0.928]","0.772 [0.721,0.822]","0.772 [0.722,0.823]"
5,"0.93 [0.894,0.967]","0.802 [0.737,0.867]","0.804 [0.739,0.869]"
ALL,"0.904 [0.888,0.92]","0.771 [0.745,0.796]","0.77 [0.745,0.796]"


# 6. Comparing to imputation

## 6.1 Save our model

In [14]:
pre_df_ci = df_ci
pre_df_p = df_p
pre_y_test, pre_y_predicted = t_y_test, t_y_predicted
pre_pid = t_pid_test

## 6.2 Preparing the imputed dataframe

In [15]:
#========================================
#  Imputation for X
#========================================
# Imputation for nan
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(key_X)
imputated_key_X = imp.transform(key_X)
# Imputation for -1
imp = IterativeImputer(max_iter=10, random_state=0, missing_values=-1)
imp.fit(imputated_key_X)
imputated_key_X = imp.transform(imputated_key_X)
# Make DataFrame
key_X = pd.DataFrame(imputated_key_X, columns=key_X.columns)


#========================================
#  Make X_y (df)
#========================================
# Last column : DEAD(=1) or ALIVE(=0)
y = eICU_file["apachePatientResult"][['patientunitstayid', 'actualicumortality']].replace('ALIVE',0).replace('EXPIRED',1)
key_X_y = pd.merge(key_X, y, on="patientunitstayid")

#========================================
# Rename
#========================================
df = key_X_y

## 6.3. Generating the model based on the imputed dataframe

In [16]:
#========================================
#  Initialization
#========================================
df_status = status(df)
df_ci = []

print("# of INPUT : ", len(df_status.ids_all), " patientunitstayids", "\n\n")
    

df_A = df

#========================================
#  Learning
#========================================    
# Prepare Trainind data and Test Data
dataset = pid_train_test_split(df_A)
X_train = dataset["X_train"]
y_train = dataset["y_train"]
X_test = dataset["X_test"]
y_test = dataset["y_test"]
pid_test = dataset["pid_test"]

# Grid Search
h_parm = grid_search(X_train, y_train)
    
# Cross Validation
cross_validation(h_parm, X_train, y_train)

# Model Generation
rdm = fitting(h_parm, X_train, y_train)

# Prediction
y_predicted = prediction(rdm, X_test)

# of INPUT :  4226  patientunitstayids 




/Users/tatsumashoji/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


0.84441 ± 0.00513


## 6.4. Comparing with our model

In [18]:
#========================================
#  Evaluation
#========================================    
# Prediction by APACHE
df_apache = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IV"].query("patientunitstayid in @ pid_test")
id_IV = df_apache[df_apache["apacheversion"]=="IV"]['patientunitstayid'].values.tolist()
y_test_IV = df_apache.query("patientunitstayid in @ id_IV").actualicumortality.replace('ALIVE',0).replace('EXPIRED',1).values.tolist()
y_predicted_IV = df_apache.query("patientunitstayid in @ id_IV").predictedicumortality.values.tolist()

df_apache = eICU_file['apachePatientResult'][eICU_file['apachePatientResult']["apacheversion"]=="IVa"].query("patientunitstayid in @ pid_test")
id_IVa = df_apache[df_apache["apacheversion"]=="IVa"]['patientunitstayid'].values.tolist()
y_test_IVa = df_apache.query("patientunitstayid in @ id_IVa").actualicumortality.replace('ALIVE',0).replace('EXPIRED',1).values.tolist()
y_predicted_IVa = df_apache.query("patientunitstayid in @ id_IVa").predictedicumortality.values.tolist()


#========================================
#  Draw all ROC
#========================================        
# Imputation
fpr_imp, tpr_imp, thresholds = metrics.roc_curve(y_test, y_predicted, drop_intermediate=False)
auc_imp = metrics.auc(fpr_imp, tpr_imp)
# Ours
fpr_ours, tpr_ours, thresholds = metrics.roc_curve(pre_y_test, pre_y_predicted, drop_intermediate=False)
auc_ours = metrics.auc(fpr_ours, tpr_ours)
# APACHE IV
fpr_IV, tpr_IV, thresholds = metrics.roc_curve(y_test_IV, y_predicted_IV, drop_intermediate=False)
auc_IV = metrics.auc(fpr_IV, tpr_IV)    
# APACHE IVa
fpr_IVa, tpr_IVa, thresholds = metrics.roc_curve(y_test_IVa, y_predicted_IVa, drop_intermediate=False)
auc_IVa = metrics.auc(fpr_IVa, tpr_IVa)    

# Ours + APACHE IV + APACHE IVa dataset for Bokeh
source = ColumnDataSource(data=dict(
    x_imp = fpr_imp,
    y_imp = tpr_imp,    
    x_ours = fpr_ours,
    y_ours = tpr_ours,    
    x_IV = fpr_IV,
    y_IV = tpr_IV,    
    x_IVa = fpr_IVa,
    y_IVa = tpr_IVa,    
))
    
# Title, Size
p = figure(
    title = 'ROC curve',
    plot_width=350,
    plot_height=350
)
    
# Axis Labels
p.xaxis.axis_label = 'False Positive Rate'
p.yaxis.axis_label = 'True Positive Rate'

# Plot
p.line('x_imp', 'y_imp', color='orange', legend='Imputation', line_width=2, source=source)
p.line('x_ours', 'y_ours', color='red', legend='Ours', line_width=2, source=source)
p.line('x_IV', 'y_IV', color='green', legend='APACHE IV', line_width=2, source=source)
p.line('x_IVa', 'y_IVa', color='blue', legend='APACHE IVa', line_width=2, source=source)

# Legend
p.legend.location = "bottom_right"
p.legend.label_text_font_style = "bold"
    
# Texts for AUC
# "AUC"
description=Label(
    x_offset=145,
    y_offset=165,
    x_units='screen',
    y_units='screen',
    text = 'AUC',
    text_font_size='1pt', render_mode="css"
)        
p.add_layout(description)    
# AUC for Ours
description=Label(
    x_offset=140,
    y_offset=150,
    x_units='screen',
    y_units='screen',
    text = '%.5f (Ours)'%auc_ours,
    text_font_size='1pt', render_mode="css"
)        
p.add_layout(description)    
# AUC for Imputation
description=Label(
    x_offset=140,
    y_offset=135,
    x_units='screen',
    y_units='screen',
    text = '%.5f (Imputation)'%auc_imp,
    text_font_size='1pt', render_mode="css"
)        
p.add_layout(description)    
# AUC for APACHE IV
description=Label(
x_offset=140,
    y_offset=120,
    x_units='screen',
    y_units='screen',
    text = '%.5f (APACHE IV)'%auc_IV,
    text_font_size='1pt', render_mode="css"
)        
p.add_layout(description)    
# AUC for APACHE IVa
description=Label(
    x_offset=140,
    y_offset=105,
    x_units='screen',
    y_units='screen',
    text = '%.5f (APACHE IVa)'%auc_IVa,
    text_font_size='1pt', render_mode="css"
)        
p.add_layout(description)    
    
# Output on the notebook
output_notebook()    
show(p)
    

Loading BokehJS ...

In [20]:
# Confidence Interval
a = str(round(auc_ours,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_ours, np.array(pre_y_test), np.array(pre_y_predicted))])+"]"
d = str(round(auc_imp,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_imp, np.array(y_test), np.array(y_predicted))])+"]"
b = str(round(auc_IV,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_IV, np.array(y_test_IV), np.array(y_predicted_IV))])+"]"
c = str(round(auc_IVa,3)) + " [" + ",".join([str(round(i,3)) for i in get_ci(auc_IVa, np.array(y_test_IVa), np.array(y_predicted_IVa))])+"]"
df_ci = [[a,d, b,c]]
df_ci

[['0.904 [0.888,0.92]',
  '0.835 [0.816,0.855]',
  '0.77 [0.744,0.796]',
  '0.768 [0.742,0.794]']]